### 데이터 로드

In [1]:
from kcycle.loader import load_data

import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, r2_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

train = load_data()

print(train.shape)

(137207, 45)


In [2]:
def clean_race_data(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    # 드롭할 컬럼
    drop_cols = [
        # '날짜',
        '경주시간', '이름', '기수', '훈련지',
        '훈련동참자', '훈련내용', '훈련일수',
        '최근3_장소일자', '최근2_장소일자', '최근1_장소일자'
    ]
    df = df.drop(columns=drop_cols, errors='ignore')

    # 광명에서 경주한 것만 필터링
    df = df[df['경주지역'] == '광명'].reset_index(drop=True)

    # 기어배수 처리
    df['기어배수'] = df['기어배수'].astype(str).str.strip().str[:4]

    # 200m 처리
    df['200m'] = df['200m'].astype(str).str.replace('"', '.', regex=False)

    # 지표관련 변수 처리
    cols = ['승률', '연대율', '삼연대율', '입상/출전', '선행', '젖히기', '추입', '마크']
    for col in cols:
        col_str = df[col].astype(str)

        # 괄호 안쪽 데이터 사용
        # inner = col_str.str.extract(r"\((.*?)\)")[0]
        # df[col] = col_str.str.extract(r"\((.*?)\)")[0]
        # df[col] = inner.fillna(col_str)

        # 괄호 바깥쪽 데이터 사용
        df[col] = col_str.str.replace(r"\(.*?\)", "", regex=True).str.strip()

    # '/'가 포함되어있는 변수 처리
    # '입상/출전' 분리
    ratio_split = df['입상/출전'].astype(str).str.extract(r"(\d+)/(\d+)")
    df['입상'] = pd.to_numeric(ratio_split[0], errors='coerce')
    df['출전'] = pd.to_numeric(ratio_split[1], errors='coerce')
    df = df.drop(columns='입상/출전')

    # '최근3순위' 계산 (소수) ('279/554' → 0.5036)
    ratio_recent = df['최근3순위'].astype(str).str.extract(r"(\d+)/(\d+)")
    a = pd.to_numeric(ratio_recent[0], errors='coerce')
    b = pd.to_numeric(ratio_recent[1], errors='coerce')
    df['최근3순위'] = a / b

    # 등급조정, 현재와 이전 등급 분리
    s = df['등급조정'].astype(str)
    df['현재_등급'] = s.str.slice(5, 7)
    df['이전_등급'] = s.str.slice(12, 14)
    df = df.drop(columns='등급조정')

    # 최근3득점: 광명과 종합 점수 분리
    s = df['최근3득점'].astype(str)
    gwang = s.str.extract(r"\(광명\)\s*([\d.]+)")[0]
    jonghap = s.str.extract(r"\(종합\)\s*([\d.]+)")[0]
    df['광명_3득점'] = pd.to_numeric(gwang, errors='coerce')
    df['종합_3득점'] = pd.to_numeric(jonghap, errors='coerce')
    df = df.drop(columns='최근3득점')

    cols = [
        '최근3_1일', '최근3_2일', '최근3_3일',
        '최근2_1일', '최근2_2일', '최근2_3일',
        '최근1_1일', '최근1_2일', '최근1_3일',
        '금회_1일', '금회_2일', '금회_3일',
    ]
    for col in cols:
        col_data = df[col].astype(str)
        pattern = r'^(\S{2})\s*(\d+)-(\d+)(\S?)'

        extracted = col_data.str.extract(pattern)
        # df[f'{col}_종류'] = extracted[0]
        df[f'{col}_순위'] = pd.to_numeric(extracted[2], errors='coerce').clip(upper=7)
        # df[f'{col}_전법'] = extracted[3]
        df = df.drop(columns=col)

    # 경주종류 단순화
    mapping = {
        '특선': '특선',
        '특별특선': '특선',
        '특선결승': '특선결승',
        '특선준결': '특선결승',
        '그랑프리결승': '특선결승',
        '우수': '우수',
        '우수결승': '우수결승',
        '우수준결': '우수결승',
        '선발': '선발',
        '선발결승': '선발결승',
        '선발준결': '선발결승',

        '준결': '특선', # 준결 경기에는 S등급 선수들이 가장 많음
        '특별': '우수', # 특별 경기에는 A등급 선수들이 가장 많음
        '특우': '특선', # 특우 경기에는 S등급 선수들이 가장 많음
        '특별우수': '우수',
    }
    df['경주종류'] = df['경주종류'].map(mapping)

    return df

train = clean_race_data(train)

### Train, Valid, Test split

In [3]:
def split_train_test_by_race(df, test_size=0.2) -> (pd.DataFrame, pd.DataFrame):
    # '경주' 단위로 train/test 분리 (한 경주에 속한 모든 선수는 같은 세트에 속하도록).
    # 미래의 경주 데이터로 과거의 경주를 예측하지 않도록 shuffle은 X.

    df = df.copy()
    df['race_id'] = (
        df['연도'].astype(str) + '_' +
        df['회차'].astype(str) + '_' +
        df['일차'].astype(str) + '_' +
        df['경주번호']
    )

    unique_races = df['race_id'].drop_duplicates().tolist()
    n = len(unique_races)
    cutoff = int(n * (1 - test_size))

    train_race_ids = set(unique_races[:cutoff])
    test_race_ids  = set(unique_races[cutoff:])

    train_df = df[df['race_id'].isin(train_race_ids)].drop(columns='race_id').reset_index(drop=True)
    test_df  = df[df['race_id'].isin(test_race_ids )].drop(columns='race_id').reset_index(drop=True)

    return train_df, test_df

train, val = split_train_test_by_race(train, test_size=0.2)
val, test = split_train_test_by_race(val, test_size=0.5)

### Data Preprocessing

In [4]:
def impute_missing_value(train: pd.DataFrame, valid: pd.DataFrame):
    # 선수가 속한 현재 등급의 평균으로 대체
    train['200m'] = pd.to_numeric(train['200m'].replace('-', pd.NA), errors='coerce')
    valid['200m'] = pd.to_numeric(valid['200m'].replace('-', pd.NA), errors='coerce')

    group_means = train.groupby('현재_등급')['200m'].mean()
    train['200m'] = train.apply(
        lambda row: group_means[row['현재_등급']] if pd.isna(row['200m']) else row['200m'],
        axis=1
    )
    valid['200m'] = valid.apply(
        lambda row: group_means.get(row['현재_등급'], pd.NA) if pd.isna(row['200m']) else row['200m'],
        axis=1
    )

    group_means = train.groupby('현재_등급')['종합_3득점'].mean()
    train['종합_3득점'] = train.apply(
        lambda row: group_means[row['현재_등급']] if pd.isna(row['종합_3득점']) else row['종합_3득점'],
        axis=1
    )
    valid['종합_3득점'] = valid.apply(
        lambda row: group_means.get(row['현재_등급'], pd.NA) if pd.isna(row['종합_3득점']) else row['종합_3득점'],
        axis=1
    )

    # 최근 순위가 결측인 경우(후보, 결장 등), 입상하지 못한 것과 동일하게 대체
    # 순위보다 입상여부, 입상을 했으면 몇등을 했는지가 중요하다고 판단
    # 1,2,3 > 순위 / 4 > 미입상
    cols = [
        '최근3_1일_순위', '최근3_2일_순위', '최근3_3일_순위',
        '최근2_1일_순위', '최근2_2일_순위', '최근2_3일_순위',
        '최근1_1일_순위', '최근1_2일_순위', '최근1_3일_순위',
        '금회_1일_순위', '금회_2일_순위', '금회_3일_순위'
    ]
    train[cols] = train[cols].fillna(7).clip(upper=4)
    valid[cols] = valid[cols].fillna(7).clip(upper=4)

    return train, valid

def drop_constant_columns(train: pd.DataFrame, valid: pd.DataFrame):
    nunique = train.nunique()
    constant_cols = nunique[nunique == 1].index.tolist()

    train = train.drop(columns=constant_cols)
    valid = valid.drop(columns=constant_cols)

    return train, valid

def drop_unused_columns(df):
    cols_to_drop = [
        '날짜', '연도', '회차', '일차', '경주번호',
    ]

    return df.drop(columns=cols_to_drop)

def encode_categorical(train: pd.DataFrame, valid: pd.DataFrame):
    cat_cols = [
        '경주종류', '번호', '현재_등급', '이전_등급',
        '최근3_1일_순위', '최근3_2일_순위', '최근3_3일_순위',
        '최근2_1일_순위', '최근2_2일_순위', '최근2_3일_순위',
        '최근1_1일_순위', '최근1_2일_순위', '최근1_3일_순위',
        '금회_1일_순위', '금회_2일_순위', '금회_3일_순위'
    ]

    for col in cat_cols:
        encoder = LabelEncoder()
        train[col] = encoder.fit_transform(train[col])
        valid[col] = encoder.transform(valid[col])

    return train, valid

def cast_features(df):
    cols = [col for col in df.columns]
    df[cols] = df[cols].apply(pd.to_numeric, errors='ignore')

    return df

def all_process(train, valid):
    train, valid = impute_missing_value(train, valid)
    train, valid = drop_constant_columns(train, valid)
    train, valid = drop_unused_columns(train), drop_unused_columns(valid)
    train, valid = encode_categorical(train, valid)
    train, valid = cast_features(train), cast_features(valid)
    return train, valid

train, val = all_process(train, val)

C:\Users\eunhak\AppData\Local\Temp\ipykernel_3280\1479108503.py:74: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[cols] = df[cols].apply(pd.to_numeric, errors='ignore')
C:\Users\eunhak\AppData\Local\Temp\ipykernel_3280\1479108503.py:74: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[cols] = df[cols].apply(pd.to_numeric, errors='ignore')


In [5]:
def add_target(df, bet_type='복승'):
    """
    bet_type:
      - '삼복승': rank <= 3  →  target=1
      - '복승':   rank <= 2  → target=1
      - '단승':   rank == 1  → target=1
    """
    df = df.copy()
    # 기준 등수 설정
    if bet_type == '삼복승':
        cutoff = 3
        df['target'] = (df['rank'] <= cutoff).astype(int)
    elif bet_type == '복승':
        cutoff = 2
        df['target'] = (df['rank'] <= cutoff).astype(int)
    elif bet_type == '단승':
        # ==1 일 때만 1
        df['target'] = (df['rank'] == 1).astype(int)
    else:
        raise ValueError(f"알 수 없는 bet_type: {bet_type!r}. ('단승','복승','삼복승' 중 하나)")

    df = df.drop(columns=['rank'])

    return df

In [6]:
## Augmentation

from joblib import Parallel, delayed
def shuffle_fixed_block(
    data: pd.DataFrame,
    per_race: int,
    ratio: float,
    exclude_back_no_list: list[int] = None,
    random_seed: int = None,
    n_jobs: int = 1
) -> pd.DataFrame:
    """
    고정 크기 블록(per_race) 단위로 데이터프레임을 샘플링·셔플합니다.
    필요하면 특정 BACK_NO 값들은 섞지 않고 원위치에 고정시킬 수 있으며,
    n_jobs로 병렬 처리도 지원합니다.

    Parameters
    ----------
    data : pd.DataFrame
        입력 데이터. 행 순서대로 per_race씩 블록화됩니다.
    per_race : int
        한 블록(경주)당 행(샘플) 수.
    ratio : float
        전체 블록 중 몇 %를 무작위로 추출해 처리할지 (0.0 ~ 1.0).
    exclude_back_no_list : list of int, optional
        이 BACK_NO 값들에 해당하는 행들은 섞지 않고 원위치에 둡니다.
    random_seed : int, optional
        랜덤 시드(재현성 확보).
    n_jobs : int, default=1
        그룹 단위 셔플을 몇 개의 프로세스로 병렬 실행할지.

    Returns
    -------
    pd.DataFrame
        선택된 블록만 섞은 뒤 합쳐서 반환합니다. RACE_ID 컬럼은 삭제됩니다.
    """
    # 1) 랜덤 시드 고정
    if random_seed is not None:
        np.random.seed(random_seed)

    # 2) RACE_ID 생성 (고정 크기 블록)
    df = data.copy()
    df['RACE_ID'] = np.arange(len(df)) // per_race

    # 3) 블록 샘플링
    race_ids = df['RACE_ID'].unique()
    n_select = int(len(race_ids) * ratio)
    selected_ids = np.random.choice(race_ids, n_select, replace=False)
    df_sampled = df[df['RACE_ID'].isin(selected_ids)]

    # 4) 그룹별 처리 함수 정의
    def _shuffle_group(group: pd.DataFrame) -> pd.DataFrame:
        gl = group.reset_index(drop=True)
        # 제외할 로컬 인덱스 계산
        if exclude_back_no_list:
            excl_pos = gl[gl['번호'].isin(exclude_back_no_list)].index.tolist()
        else:
            excl_pos = []
        # 나머지 행들만 완전 셔플
        rest = gl.drop(index=excl_pos)
        rest_shuffled = rest.sample(frac=1.0, random_state=random_seed).reset_index(drop=True)
        # 빈 슬롯(로컬 인덱스)
        slots = [i for i in range(per_race) if i not in excl_pos]
        # 새 그룹 생성
        new_gl = gl.copy()
        for i, slot in enumerate(slots):
            new_gl.iloc[slot] = rest_shuffled.iloc[i]
        return new_gl

    # 5) 병렬 혹은 순차 처리
    groups = [grp for _, grp in df_sampled.groupby('RACE_ID')]
    if n_jobs == 1:
        shuffled = [_shuffle_group(g) for g in groups]
    else:
        shuffled = Parallel(n_jobs=n_jobs)(
            delayed(_shuffle_group)(g) for g in groups
        )

    # 6) 결과 합치고 정리
    result = pd.concat(shuffled, ignore_index=True)
    return result.drop(columns=['RACE_ID'])


def reverse_fixed_block(
    data: pd.DataFrame,
    per_race: int,
    ratio: float,
    n_jobs: int = 1
) -> pd.DataFrame:
    """
    고정 크기 블록(per_race) 단위로 순서를 뒤집어 샘플링합니다.

    Parameters
    ----------
    data : pd.DataFrame
        입력 데이터. 행 순서대로 per_race씩 블록화됩니다.
    per_race : int
        한 블록(경주)당 행(샘플) 수.
    ratio : float
        전체 블록 중 몇 %를 무작위로 선택해 순서를 뒤집을지 (0.0 ~ 1.0).
    n_jobs : int, default=1
        블록 단위 처리를 몇 개의 프로세스로 병렬 실행할지.

    Returns
    -------
    pd.DataFrame
        선택된 블록만 순서를 뒤집어 합쳐서 반환합니다. RACE_ID 컬럼은 삭제됩니다.
    """
    # 1) 랜덤 시드 고정

    # 2) RACE_ID 생성 (고정 크기 블록)
    df = data.copy()
    df['RACE_ID'] = np.arange(len(df)) // per_race

    # 3) 블록 샘플링
    race_ids = df['RACE_ID'].unique()
    n_select = int(len(race_ids) * ratio)
    selected_ids = np.random.choice(race_ids, n_select, replace=False)
    df_sampled = df[df['RACE_ID'].isin(selected_ids)]

    # 4) 그룹별 뒤집기 함수
    def _reverse_group(group: pd.DataFrame) -> pd.DataFrame:
        gl = group.reset_index(drop=True)
        return gl.iloc[::-1]

    # 5) 병렬 또는 순차 처리
    groups = [grp for _, grp in df_sampled.groupby('RACE_ID')]
    if n_jobs == 1:
        reversed_groups = [_reverse_group(g) for g in groups]
    else:
        reversed_groups = Parallel(n_jobs=n_jobs)(
            delayed(_reverse_group)(g) for g in groups
        )

    # 6) 결과 합치고 정리
    result = pd.concat(reversed_groups, ignore_index=True)
    return result.drop(columns=['RACE_ID'])

In [150]:
train = load_data()
train = clean_race_data(train)
train, val = split_train_test_by_race(train, test_size=0.2)
val, test = split_train_test_by_race(val, test_size=0.5)
train_ = train.copy()

train, val = all_process(train, val)
_, test = all_process(train_, test)

print(train.shape, val.shape, test.shape)

C:\Users\eunhak\AppData\Local\Temp\ipykernel_3280\1479108503.py:74: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[cols] = df[cols].apply(pd.to_numeric, errors='ignore')
C:\Users\eunhak\AppData\Local\Temp\ipykernel_3280\1479108503.py:74: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[cols] = df[cols].apply(pd.to_numeric, errors='ignore')
C:\Users\eunhak\AppData\Local\Temp\ipykernel_3280\1479108503.py:74: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[cols] = df[cols].apply(pd.to_numeric, errors='ignore')


(88368, 32) (11046, 32) (11046, 32)


C:\Users\eunhak\AppData\Local\Temp\ipykernel_3280\1479108503.py:74: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[cols] = df[cols].apply(pd.to_numeric, errors='ignore')


In [151]:
# 필요시 증강

train_augmented_shuffle = shuffle_fixed_block(
    train, per_race=7, ratio=0.5, exclude_back_no_list=[3], random_seed=42, n_jobs=12,
)
train_augmented_reverse = reverse_fixed_block(
    train, per_race=7, ratio=0.5, n_jobs=12,
)

print(f'기존 학습 데이터: {train.shape[0]}, 증강한 데이터: {train_augmented_shuffle.shape[0]} + {train_augmented_reverse.shape[0]}')

n_aug = len(train_augmented_shuffle)
train_augmented_shuffle['번호'] = train['번호'].iloc[:n_aug].values

n_aug = len(train_augmented_reverse)
train_augmented_reverse['번호'] = train['번호'].iloc[:n_aug].values

train = pd.concat([train, train_augmented_shuffle, train_augmented_reverse], ignore_index=True).reset_index(drop=True)

print(f'최종 학습 데이터: {train.shape[0]}개')

기존 학습 데이터: 88368, 증강한 데이터: 44184 + 44184
최종 학습 데이터: 176736개


In [152]:
bet_type = '삼복승'  # 단승, 복승, 삼복승

if bet_type=='단승':
    top_k = 1
elif bet_type=='복승':
    top_k = 2
elif bet_type=='삼복승':
    top_k = 3

train = add_target(train, bet_type=bet_type)
val = add_target(val, bet_type=bet_type)
test = add_target(test, bet_type=bet_type)

### Model Training

In [168]:
X_train = train.drop(columns=['target'])
X_val = val.drop(columns=['target'])
X_test = test.drop(columns=['target'])

y_train = train['target']
y_val = val['target']
y_test = test['target']

cat_cols = [
    '경주종류', '번호', '현재_등급', '이전_등급',
    '최근3_1일_순위', '최근3_2일_순위', '최근3_3일_순위',
    '최근2_1일_순위', '최근2_2일_순위', '최근2_3일_순위',
    '최근1_1일_순위', '최근1_2일_순위', '최근1_3일_순위',
    '금회_1일_순위', '금회_2일_순위', '금회_3일_순위'
]
X_train[cat_cols] = X_train[cat_cols].astype('category')
X_val[cat_cols] = X_val[cat_cols].astype('category')
X_test[cat_cols] = X_test[cat_cols].astype('category')

### Random Forest
# model = RandomForestClassifier(
#     n_estimators=1000,
#     random_state=42,
#     n_jobs=-1,
# )
# model.fit(X_train, y_train)
###

### LightGBM
# model = LGBMClassifier(
#     n_estimators=5000,
#     random_state=42,
#     enable_categorical=True,
#     early_stopping_rounds=100,
#     verbose=-1,
# )
# model.fit(
#     X_train, y_train,
#     eval_set=(X_val, y_val),
#     categorical_feature=cat_cols
# )
###

### XGBoost
# model = XGBClassifier(
#     n_estimators=5000,
#     random_state=42,
#     enable_categorical=True,
#     early_stopping_rounds=100,
# )
# model.fit(
#     X_train, y_train,
#     eval_set=[(X_val, y_val)],
#     verbose=0,
# )
###

### Logistic Regression
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

model = LogisticRegression(
    random_state=42,
    max_iter=1000,
    n_jobs=-1,
)
model.fit(
    X_train, y_train,
)
###

y_train_pred = model.predict_proba(X_train)[:, 1]
y_val_pred = model.predict_proba(X_val)[:, 1]
y_test_pred = model.predict_proba(X_test)[:, 1]

### Evaluation

- 개별 지표

In [169]:
# Train set metrics
train_accuracy = accuracy_score(y_train, (y_train_pred > 0.5).astype(int))
train_auc = roc_auc_score(y_train, y_train_pred)
train_f1 = f1_score(y_train, (y_train_pred > 0.5).astype(int))
train_r2 = r2_score(y_train, y_train_pred)

# Validation set metrics 
val_accuracy = accuracy_score(y_val, (y_val_pred > 0.5).astype(int))
val_auc = roc_auc_score(y_val, y_val_pred)
val_f1 = f1_score(y_val, (y_val_pred > 0.5).astype(int))
val_r2 = r2_score(y_val, y_val_pred)

# Test set metrics
test_accuracy = accuracy_score(y_test, (y_test_pred > 0.5).astype(int))
test_auc = roc_auc_score(y_test, y_test_pred)
test_f1 = f1_score(y_test, (y_test_pred > 0.5).astype(int))
test_r2 = r2_score(y_test, y_test_pred)

metrics_df = pd.DataFrame({
    'Train': [train_accuracy, train_auc, train_f1, train_r2],
    'Validation': [val_accuracy, val_auc, val_f1, val_r2],
    'Test': [test_accuracy, test_auc, test_f1, test_r2]
}, index=['Accuracy', 'AUC', 'F1', 'R2'])

metrics_df.round(3)

,Train,Validation,Test
Accuracy,0.742,0.716,0.723
AUC,0.799,0.782,0.790
F1,0.678,0.660,0.667
R2,0.273,0.195,0.221


- 경주별 지표

In [170]:
def reshape_by_race(arr: np.ndarray, per_race: int = 7) -> np.ndarray:
    """
    - 1D 배열 (n_samples,)   → (n_races, per_race)
    - 2D 배열 (n_samples, f) → (n_races, per_race * f)
    """
    arr = np.asarray(arr)

    # 1D
    if arr.ndim == 1:
        total = arr.size
        if total % per_race != 0:
            raise ValueError(f"배열 길이({total})가 per_race({per_race})의 배수가 아닙니다.")
        return arr.reshape(-1, per_race)

    # 2D
    elif arr.ndim == 2:
        n_samples, n_features = arr.shape
        if n_samples % per_race != 0:
            raise ValueError(f"첫 번째 축 길이({n_samples})가 per_race({per_race})의 배수가 아닙니다.")
        n_races = n_samples // per_race
        # per_race 개의 샘플 × n_features를 한 축으로 합침
        return arr.reshape(n_races, per_race * n_features)

    else:
        raise ValueError(f"지원하지 않는 차원(ndim={arr.ndim})입니다. 1D 또는 2D만 처리 가능합니다.")

# y_test 배열을 (N,)에서 (N/7, 7)로 변환
y_train_race = reshape_by_race(y_train, per_race=7)
y_train_pred_race = reshape_by_race(y_train_pred, per_race=7)

y_val_race = reshape_by_race(y_val, per_race=7)
y_val_pred_race = reshape_by_race(y_val_pred, per_race=7)

y_test_race = reshape_by_race(y_test, per_race=7)
y_test_pred_race = reshape_by_race(y_test_pred, per_race=7)

print(y_train_race.shape, y_train_pred_race.shape)
print(y_val_race.shape, y_val_pred_race.shape)
print(y_test_race.shape, y_test_pred_race.shape)

(25248, 7) (25248, 7)
(1578, 7) (1578, 7)
(1578, 7) (1578, 7)


In [171]:
def compute_race_metrics(
    y_true: np.ndarray,
    y_score: np.ndarray,
    threshold: float = 0.5,
    top_k: int = 2
) -> pd.Series:
    """
    경주별 multi‐label 지표를 계산합니다.
    입력 y_true, y_score는 각각 (n_races, n_classes) 형태여야 합니다.

    race_accuracy는 per‐race 내 top_k만 positive로 간주했을 때,
    그 이진 예측이 true label과 완전히 일치한 레이스의 비율입니다.

    Parameters
    ----------
    y_true : array-like, shape (n_races, n_classes)
        실제 binary 타깃 매트릭스 (0/1).
    y_score : array-like, shape (n_races, n_classes)
        예측 점수(확률) 매트릭스.
    threshold : float, default=0.5
        multi‐label 지표를 위한 이진화 기준.
    top_k : int, default=2
        race_accuracy 계산 시 per‐race 내 상위 k개만 positive로 간주.

    Returns
    -------
    pd.Series
        {
            "race_accuracy": float
        }
    """
    y_true = np.asarray(y_true)
    y_score = np.asarray(y_score)
    if y_true.shape != y_score.shape:
        raise ValueError(f"y_true.shape {y_true.shape}와 y_score.shape {y_score.shape}가 일치해야 합니다.")
    if y_true.ndim != 2:
        raise ValueError(f"입력 배열은 2D여야 합니다. (현재 ndim={y_true.ndim})")

    # 1) threshold 기반 이진화 (multi‐label 지표용)
    y_pred_thr = (y_score > threshold).astype(int)

    # 2) top_k 기반 이진화 (race_accuracy용)
    n_races, n_classes = y_score.shape
    y_pred_topk = np.zeros_like(y_pred_thr, dtype=int)
    for i in range(n_races):
        # 상위 k개 인덱스
        top_idx = np.argsort(y_score[i])[-top_k:]
        y_pred_topk[i, top_idx] = 1

    # 3) race_accuracy: per‐race 내 top_k 예측이 true와 완전 일치한 비율
    race_acc = np.mean(np.all(y_true == y_pred_topk, axis=1))

    return pd.Series({"race_accuracy": race_acc})


# 단승: top_k=1, 복승:top_k=2, 삼복승:top_k=3
train_metrics = compute_race_metrics(y_train_race, y_train_pred_race, top_k=top_k)
val_metrics   = compute_race_metrics(y_val_race, y_val_pred_race, top_k=top_k)
test_metrics  = compute_race_metrics(y_test_race, y_test_pred_race, top_k=top_k)

metrics_df = pd.DataFrame({
    "Train": train_metrics,
    "Validation": val_metrics,
    "Test": test_metrics
})

metrics_df.round(3)

,Train,Validation,Test
race_accuracy,0.304,0.264,0.26


### 경주인 것을 고려한 모델링

In [204]:
train = load_data()
train = clean_race_data(train)
train, val = split_train_test_by_race(train, test_size=0.2)
val, test = split_train_test_by_race(val, test_size=0.5)
train_ = train.copy()

train, val = all_process(train, val)
_, test = all_process(train_, test)

print(train.shape, val.shape, test.shape)

C:\Users\eunhak\AppData\Local\Temp\ipykernel_3280\1479108503.py:74: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[cols] = df[cols].apply(pd.to_numeric, errors='ignore')
C:\Users\eunhak\AppData\Local\Temp\ipykernel_3280\1479108503.py:74: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[cols] = df[cols].apply(pd.to_numeric, errors='ignore')
C:\Users\eunhak\AppData\Local\Temp\ipykernel_3280\1479108503.py:74: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[cols] = df[cols].apply(pd.to_numeric, errors='ignore')


(88368, 32) (11046, 32) (11046, 32)


C:\Users\eunhak\AppData\Local\Temp\ipykernel_3280\1479108503.py:74: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[cols] = df[cols].apply(pd.to_numeric, errors='ignore')


In [205]:
## 필요시 증강

# 3번 라인을 제외하고 shuffle
train_augmented_shuffle = shuffle_fixed_block(
    train, per_race=7, ratio=0.5, exclude_back_no_list=[3], random_seed=42, n_jobs=12,
)

# 1번부터 7번까지의 순서를 뒤집음(3번 라인은 그대로)
train_augmented_reverse = reverse_fixed_block(
    train, per_race=7, ratio=0.5, n_jobs=12,
)

print(f'기존 학습 데이터: {train.shape[0]}, 증강한 데이터: {train_augmented_shuffle.shape[0]} + {train_augmented_reverse.shape[0]}')

n_aug = len(train_augmented_shuffle)
train_augmented_shuffle['번호'] = train['번호'].iloc[:n_aug].values

n_aug = len(train_augmented_reverse)
train_augmented_reverse['번호'] = train['번호'].iloc[:n_aug].values

train = pd.concat([train, train_augmented_shuffle, train_augmented_reverse], ignore_index=True).reset_index(drop=True)

print(f'최종 학습 데이터: {train.shape[0]}개')

기존 학습 데이터: 88368, 증강한 데이터: 44184 + 44184
최종 학습 데이터: 176736개


In [206]:
# '번호' 컬럼은 reshape하면서 번호에 맞게 위치가 고정되므로 drop
train = train.drop(columns=['번호'])
val = val.drop(columns=['번호'])
test = test.drop(columns=['번호'])

print(train.shape, val.shape, test.shape)

bet_type = '삼복승'  # 단승, 복승, 삼복승

if bet_type=='단승':
    top_k = 1
elif bet_type=='복승':
    top_k = 2
elif bet_type=='삼복승':
    top_k = 3

train = add_target(train, bet_type=bet_type)
val = add_target(val, bet_type=bet_type)
test = add_target(test, bet_type=bet_type)

(176736, 31) (11046, 31) (11046, 31)


In [207]:
X_train = train.drop(columns=['target'])
X_val = val.drop(columns=['target'])
X_test = test.drop(columns=['target'])

y_train = train['target']
y_val = val['target']
y_test = test['target']

cat_cols = [
    '경주종류', '현재_등급', '이전_등급',
    '최근3_1일_순위', '최근3_2일_순위', '최근3_3일_순위',
    '최근2_1일_순위', '최근2_2일_순위', '최근2_3일_순위',
    '최근1_1일_순위', '최근1_2일_순위', '최근1_3일_순위',
    '금회_1일_순위', '금회_2일_순위', '금회_3일_순위'
]
X_train[cat_cols] = X_train[cat_cols].astype('category')
X_val[cat_cols] = X_val[cat_cols].astype('category')
X_test[cat_cols] = X_test[cat_cols].astype('category')

print(X_train.shape, X_val.shape, X_test.shape)

X_train_race = reshape_by_race(X_train, per_race=7)
X_val_race = reshape_by_race(X_val, per_race=7)
X_test_race = reshape_by_race(X_test, per_race=7)

y_train_race = reshape_by_race(y_train, per_race=7)
y_val_race = reshape_by_race(y_val, per_race=7)
y_test_race = reshape_by_race(y_test, per_race=7)

print(X_train_race.shape, X_val_race.shape, X_test_race.shape)

(176736, 30) (11046, 30) (11046, 30)
(25248, 210) (1578, 210) (1578, 210)


In [208]:
# per_race 값 (reshape_by_race에 사용한 것과 동일해야 합니다)
per_race = 7

# 전체 피처 수
n_features = X_train.shape[1]

# 1) 원본 DataFrame에서 각 cat_cols의 컬럼 인덱스 추출
orig_cat_idx = [X_train.columns.get_loc(col) for col in cat_cols]

# 2) per_race 블록별로 오프셋을 더해서 모든 인덱스 생성
cat_feature_indices = [
    block * n_features + idx
    for block in range(per_race)
    for idx in orig_cat_idx
]

cat_feature_indices = sorted(cat_feature_indices)
print(cat_feature_indices)

[0, 14, 15, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 44, 45, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 74, 75, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 104, 105, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 134, 135, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 164, 165, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 194, 195, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209]


In [209]:
from sklearn.base import BaseEstimator, ClassifierMixin, clone

class MultiOutputClassifierCustom(BaseEstimator, ClassifierMixin):
    def __init__(self, estimator):
        self.estimator = estimator

    def fit(self, X, y, eval_set=None, **fit_kwargs):
        """
        각 출력(y[:, i])마다 estimator를 복제하여 학습합니다.

        Parameters
        ----------
        X : array-like, shape (n_samples, n_features)
        y : array-like, shape (n_samples,) or (n_samples, n_outputs)
        eval_set : list of (X_val, y_val) tuples, optional
            모든 타깃에 공통으로 사용할 검증용 데이터. 각 y_val은 1D 또는 2D여도 무방하며,
            내부에서 타깃별로 분리되어 전달됩니다.
        **fit_kwargs : dict
            estimator.fit에 추가로 전달할 인자들 (예: early_stopping_rounds, eval_metric 등)
        """
        X = np.asarray(X)
        y = np.asarray(y)
        if y.ndim == 1:
            y = y.reshape(-1, 1)

        # 준비된 eval_set이 있으면, 타깃별로 분리
        if eval_set is not None:
            # eval_set 내부의 y_val 배열도 numpy로
            eval_sets_per_output = []
            for i in range(y.shape[1]):
                single_output_sets = []
                for X_val, y_val in eval_set:
                    y_val = np.asarray(y_val)
                    # 1D → 2D면, 각 열 i 분리
                    if y_val.ndim == 2:
                        y_val_i = y_val[:, i]
                    else:
                        y_val_i = y_val
                    single_output_sets.append((X_val, y_val_i))
                eval_sets_per_output.append(single_output_sets)
        else:
            eval_sets_per_output = [None] * y.shape[1]

        self.estimators_ = []
        for i in range(y.shape[1]):
            est = clone(self.estimator)
            # 이 타깃에만 해당하는 eval_set
            params = fit_kwargs.copy()
            if eval_sets_per_output[i] is not None:
                params['eval_set'] = eval_sets_per_output[i]
            est.fit(X, y[:, i], **params)
            self.estimators_.append(est)

        return self


    def predict(self, X):
        """
        학습된 각 estimator로부터 예측(label)을 얻어 합칩니다.

        Returns
        -------
        preds : ndarray, shape (n_samples, n_outputs)
        """
        X = np.asarray(X)
        # [(n_samples,), ...] → (n_outputs, n_samples)
        preds = [est.predict(X) for est in self.estimators_]
        return np.vstack(preds).T

    def predict_proba(self, X):
        """
        학습된 각 estimator의 predict_proba에서 클래스=1 확률만 추출하여 합칩니다.

        Returns
        -------
        probas : ndarray, shape (n_samples, n_outputs)
            probas[i, j]는 i번째 샘플의 j번째 타깃이 1일 확률
        """
        X = np.asarray(X)
        probas = []
        for est in self.estimators_:
            p = est.predict_proba(X)
            # binary classifier라면 p.shape = (n_samples, 2)
            if p.shape[1] != 2:
                raise ValueError("각 estimator는 이진 분류를 지원해야 합니다.")
            probas.append(p[:, 1])
        # [(n_samples,), ...] → (n_outputs, n_samples) → transpose
        return np.vstack(probas).T

In [225]:
### Random Forest
# base_model = RandomForestClassifier(
#     n_estimators=1000,
#     random_state=42,
#     n_jobs=-1,
# )
# model = MultiOutputClassifierCustom(base_model)
# model.fit(X_train_race, y_train_race)
###

### LightGBM
# base_model = LGBMClassifier(
#     n_estimators=5000,
#     random_state=42,
#     enable_categorical=True,
#     early_stopping_rounds=100,
#     verbose=-1,
# )
# model = MultiOutputClassifierCustom(base_model)
# model.fit(
#     X_train_race, y_train_race,
#     eval_set=[(X_val_race, y_val_race)],
#     categorical_feature=cat_feature_indices,
# )
###

### XGBoost
### LGBM은 categorical_feature로 범주형 변수를 지정할 수 있는데 XGBoost는 내부적으로 category타입인 것을 자동으로 처리하는듯
# col_names = [f"{col}_p{p}"
#              for p in range(per_race)
#              for col in X_train.columns]
# X_df = pd.DataFrame(X_train_race, columns=col_names)
#
# cat_cols_df = [X_df.columns[i] for i in cat_feature_indices]
# for col in cat_cols_df:
#     X_df[col] = X_df[col].astype('category')
#
# base_model = XGBClassifier(
#     n_estimators=5000,
#     random_state=42,
#     enable_categorical=True,
#     early_stopping_rounds=100,
#
# )
# model = MultiOutputClassifierCustom(base_model)
# model.fit(
#     X_df, y_train_race,
#     eval_set=[(X_val_race, y_val_race)],
#     verbose=0,
# )
###

### Logistic Regression
scaler = StandardScaler()
X_train_race = scaler.fit_transform(X_train_race)
X_val_race = scaler.transform(X_val_race)
X_test_race = scaler.transform(X_test_race)

base_model = LogisticRegression(
    random_state=42,
    max_iter=1000,
    n_jobs=-1,
)
model = MultiOutputClassifierCustom(base_model)
model.fit(X_train_race, y_train_race)
###

MultiOutputClassifierCustom(estimator=LogisticRegression(max_iter=1000,
                                                         n_jobs=-1,
                                                         random_state=42))

In [226]:
y_train_pred_race = model.predict_proba(X_train_race)
y_val_pred_race = model.predict_proba(X_val_race)
y_test_pred_race = model.predict_proba(X_test_race)

print(y_train_pred_race.shape, y_val_pred_race.shape, y_test_pred_race.shape)

(25248, 7) (1578, 7) (1578, 7)


In [227]:
# 단승: top_k=1, 복승:top_k=2, 삼복승:top_k=3
train_metrics = compute_race_metrics(y_train_race, y_train_pred_race, threshold=0.5, top_k=top_k)
val_metrics   = compute_race_metrics(y_val_race,   y_val_pred_race,   threshold=0.5, top_k=top_k)
test_metrics  = compute_race_metrics(y_test_race,  y_test_pred_race,  threshold=0.5, top_k=top_k)

metrics_df = pd.DataFrame({
    "Train": train_metrics,
    "Validation": val_metrics,
    "Test": test_metrics
})

metrics_df.round(3)

,Train,Validation,Test
race_accuracy,0.311,0.25,0.272


In [228]:
def invert_reshape_by_race(arr: np.ndarray, per_race: int = 7) -> np.ndarray:
    """
    reshape_by_race의 역함수.

    - 입력 배열 arr의 shape이 (n_races, per_race)라면
      1D 벡터 (n_races * per_race,)로 복원합니다.

    - 입력 배열 arr의 shape이 (n_races, per_race * n_features)라면
      2D 행렬 (n_races * per_race, n_features)로 복원합니다.

    Parameters
    ----------
    arr : np.ndarray, shape (n_races, M)
        reshape_by_race의 출력 배열.
    per_race : int, default=7
        한 경주당 샘플 수. reshape_by_race와 동일한 값을 써야 합니다.

    Returns
    -------
    np.ndarray
        원래 배열 형태로 복원한 결과:
        - 원본이 1D였으면 1D,
        - 원본이 2D였으면 2D.
    """
    arr = np.asarray(arr)
    if arr.ndim != 2:
        raise ValueError(f"입력 배열은 2D여야 합니다. (현재 ndim={arr.ndim})")

    n_races, M = arr.shape

    # 1D 원본 복원 (열 수가 per_race와 같을 때)
    if M == per_race:
        return arr.reshape(-1)

    # 2D 원본 복원
    if M % per_race != 0:
        raise ValueError(f"열 수({M})가 per_race({per_race})의 배수가 아닙니다.")

    n_features = M // per_race
    return arr.reshape(n_races * per_race, n_features)


y_train_pred = invert_reshape_by_race(y_train_pred_race, per_race=7)
y_val_pred = invert_reshape_by_race(y_val_pred_race, per_race=7)
y_test_pred = invert_reshape_by_race(y_test_pred_race, per_race=7)

print(y_train_pred.shape, y_val_pred.shape, y_test_pred.shape)

(176736,) (11046,) (11046,)


In [229]:
# Train set metrics
train_accuracy = accuracy_score(y_train, (y_train_pred > 0.5).astype(int))
train_auc = roc_auc_score(y_train, y_train_pred)
train_f1 = f1_score(y_train, (y_train_pred > 0.5).astype(int))
train_r2 = r2_score(y_train, y_train_pred)

# Validation set metrics
val_accuracy = accuracy_score(y_val, (y_val_pred > 0.5).astype(int))
val_auc = roc_auc_score(y_val, y_val_pred)
val_f1 = f1_score(y_val, (y_val_pred > 0.5).astype(int))
val_r2 = r2_score(y_val, y_val_pred)

# Test set metrics
test_accuracy = accuracy_score(y_test, (y_test_pred > 0.5).astype(int))
test_auc = roc_auc_score(y_test, y_test_pred)
test_f1 = f1_score(y_test, (y_test_pred > 0.5).astype(int))
test_r2 = r2_score(y_test, y_test_pred)

metrics_df = pd.DataFrame({
    'Train': [train_accuracy, train_auc, train_f1, train_r2],
    'Validation': [val_accuracy, val_auc, val_f1, val_r2],
    'Test': [test_accuracy, test_auc, test_f1, test_r2]
}, index=['Accuracy', 'AUC', 'F1', 'R2'])

metrics_df.round(3)

,Train,Validation,Test
Accuracy,0.767,0.739,0.749
AUC,0.836,0.809,0.818
F1,0.708,0.686,0.700
R2,0.342,0.251,0.273
